In [1]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support,classification_report,confusion_matrix,ConfusionMatrixDisplay, plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
import pickle5 as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder

In [2]:
#Loading in data
df = pd.read_csv("Medical_No_Shows.csv")
df.head()

,PatientID,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,LocationID,MedicaidIND,Hypertension,Diabetes,Alcoholism,Disability,SMS_received,No-show
0,#29872499824296,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,40,0,1,0,0,0,0,No
1,#558997776694438,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,40,0,0,0,0,0,0,No
2,#4262962299951,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,47,0,0,0,0,0,0,No
3,#867951213174,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,55,0,0,0,0,0,0,No
4,#8841186448183,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,40,0,1,1,0,0,0,No


### * no null-values 
### * target variable is imbalanced


In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Initial glance to check range, value counts, NAs
df.info()
df['No-show'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PatientID       110527 non-null  int32         
 1   AppointmentID   110527 non-null  int64         
 2   Gender          110527 non-null  object        
 3   ScheduledDay    110527 non-null  datetime64[ns]
 4   AppointmentDay  110527 non-null  datetime64[ns]
 5   Age             110527 non-null  int64         
 6   LocationID      110527 non-null  int64         
 7   MedicaidIND     110527 non-null  int64         
 8   Hypertension    110527 non-null  int64         
 9   Diabetes        110527 non-null  int64         
 10  Alcoholism      110527 non-null  int64         
 11  Disability      110527 non-null  int64         
 12  SMS_received    110527 non-null  int64         
 13  No-show         110527 non-null  int32         
dtypes: datetime64[ns](2), int32(2), int6

0    88208
1    22319
Name: No-show, dtype: int64

In [4]:
df.describe()

,AppointmentID,Age,LocationID,MedicaidIND,Hypertension,Diabetes,Alcoholism,Disability,SMS_received
count,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,5.675305e+06,37.088874,42.413076,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,7.129575e+04,23.110205,21.885394,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,5.030230e+06,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.640286e+06,18.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.680573e+06,37.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.725524e+06,55.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,5.790484e+06,115.000000,81.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


# Checking for out-of-ordinary values (bad data)

In [5]:
df.Gender.value_counts()
df.Age.value_counts()
df.Hypertension.value_counts()
df.Diabetes.value_counts()
df.Alcoholism.value_counts()   
df.Disability.value_counts()
df.SMS_received.value_counts()
df.MedicaidIND.value_counts()
df.LocationID.value_counts()

F    71840
M    38687
Name: Gender, dtype: int64

 0      3539
 1      2273
 52     1746
 49     1652
 53     1651
        ... 
 115       5
 100       4
 102       2
 99        1
-1         1
Name: Age, Length: 104, dtype: int64

0    88726
1    21801
Name: Hypertension, dtype: int64

0    102584
1      7943
Name: Diabetes, dtype: int64

0    107167
1      3360
Name: Alcoholism, dtype: int64

0    108286
1      2042
2       183
3        13
4         3
Name: Disability, dtype: int64

0    75045
1    35482
Name: SMS_received, dtype: int64

0    99666
1    10861
Name: MedicaidIND, dtype: int64

39    7717
45    5805
60    4431
40    3877
37    3514
      ... 
32      35
33      10
1        8
35       2
52       1
Name: LocationID, Length: 81, dtype: int64

### Disability is not suppose to be > 1 according to dictionary. When Disability is > 1, the percent of no show does not seem to vary from the overall no show percentage. We wiil drop Disability is > 1 since only 0.1% of the data will be dropped

In [6]:
df.loc[df['Disability']>1,"No-show"].value_counts()
print("no show percent for disability > 1")
41/158
print("Overall no show percent")
22319/88208

print("percent that will be dropped")
199/(22319+88208)

No     158
Yes     41
Name: No-show, dtype: int64

no show percent for disability > 1


0.25949367088607594

Overall no show percent


0.25302693633230544

percent that will be dropped


0.0018004650447401992

# Cleaning

In [7]:
#data cleaning and transformations
#change target to binary integers
df["No-show"] = (df["No-show"] == "Yes").astype("int")

#Use label_encoder to encode PatientID

labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df['PatientID'] = labelencoder.fit_transform(df['PatientID'])
#convert date to datetime
df["ScheduledDay"] = pd.to_datetime(df["ScheduledDay"], 
 format = "%Y-%m-%dT%H:%M:%SZ", 
 errors = "coerce")
df["AppointmentDay"] = pd.to_datetime(df["AppointmentDay"], 
 format = "%Y-%m-%dT%H:%M:%SZ", 
 errors = "coerce")



In [9]:
#Any scheduledDay greater than AppointmentDay?
(df['ScheduledDay'] > df['AppointmentDay']).sum()
#shift all the appointment times to the end of the day
df["AppointmentDay"] = df["AppointmentDay"] + pd.Timedelta("1d") - pd.Timedelta("1s")

38568

In [10]:
#only keep if disability <=1 as defined by dictionary. Justification given above
df = df[df['Disability']<=1] 
#Drop the one -1 age
df = df[df['Age']>-1] 
#Drop the 5 rows where scheduleDay time is after appointmentDay

In [11]:
df.to_csv('cleaned_data.csv', index = False, header = True)